# Grocery Forecasting - Kaggle

## Load dependencies

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
# from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = "data/grocery-sales/"
!ls {PATH}

holidays_events.csv  oil.csv		    stores.csv	train.csv
items.csv	     sample_submission.csv  test.csv	transactions.csv


## Read data

In [13]:
types = {'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

In [ ]:
%%time
df_all = pd.read_csv(f'{PATH}train.csv', parse_dates=['date'], dtype=types, infer_datetime_format=True)

In [ ]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False' : False, 'True' : True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

%time df_all.to_feather('tmp/raw_groceries')

In [ ]:
%time df_all.describe(include='all')

In [ ]:
df_test = pd.read_csv(f'{PATH}train.csv', parse_dates=['date'], dtype=types, infer_datetime_format=True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_all.onpromotion.map({'False' : False, 'True' : True})
df_test.onpromotion = df_all.onpromotion.astype(bool)

df_test.describe(include='all')

In [ ]:
df_all.tail()

In [ ]:
df_all = pd.read_feather('tmp/raw_groceries')

In [ ]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [ ]:
%time add_datepart(df_all, 'date')

In [4]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [ ]:
n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

In [ ]:
%%time
trn, y = proc_df(train, 'unit_sales')
val, y_val = proc_df(valid, 'unit_sales')

## Models

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val), m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
set_rf_samples(1_000_000)

In [ ]:
%time x = np.array(trn, dtype=np.float32)

In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=8)
%time m.fit(x,y)

In [ ]:
print_score(m)